In [166]:
import numpy as np

from macd_strategy2 import MACDandEMA

from quantfreedom.enums import DynamicOrderSettingsArrays, CandleBodyType
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol='BTCUSDT',
    timeframe='1h',
    candles_to_dl=10000
    )

In [168]:
candles

array([[1686711600000.00, 25977.00, 25981.50, 25959.50, 25969.50, 0.34],
       [1686715200000.00, 25969.50, 25998.00, 25951.00, 25968.00, 2.00],
       [1686718800000.00, 25968.00, 25978.00, 25820.00, 25831.00, 4.30],
       ...,
       [1713153600000.00, 65106.90, 65147.60, 64646.10, 64873.00, 513.58],
       [1713157200000.00, 64873.00, 65492.70, 64870.50, 65435.00, 438.30],
       [1713160800000.00, 65435.00, 66522.60, 65317.90, 66430.80,
        1205.37]])

In [169]:
macd_below = 30
fast_length = 12
slow_length = 26
signal_smoothing = 9
ema_length = 200

In [170]:
macd_strat = MACDandEMA(
    ema_length=np.arange(100, 501, 100),
    fast_length=np.arange(10, 61, 10),
    long_short= "long",
    macd_below=np.arange(75, 500, 25),
    signal_smoothing=np.arange(5, 16, 5),
    slow_length=np.arange(30, 121, 30),
)
macd_strat

In [171]:
backtest_settings = BacktestSettings()
    
exchange_settings = ExchangeSettings(
    asset_tick_step=3,                          # Decimal precision for Bitcoin can be high since it can be priced very granularly
    leverage_mode=1,                            # Isolated leverage is safer for managing risk in volatile markets
    leverage_tick_step=2,                       # Allows for fine control over leverage adjustments
    limit_fee_pct=0.0003,                       # Typical limit order fee percentage for lower volume tiers on Kraken
    market_fee_pct=0.0006,                      # Typical market order fee percentage for lower volume tiers on Kraken
    max_asset_size=100.0,                       # Practical upper limit for the size of a single Bitcoin order
    max_leverage=150.0,                         # Maximum allowable leverage for Bitcoin; check current regulatory limits
    min_asset_size=0.001,                       # Minimum trade size for Bitcoin on Kraken
    min_leverage=1.0,                           # Minimum leverage available (effectively no leverage)
    mmr_pct=0.004,                              # Maintenance margin requirement at 4% is typical for Bitcoin
    position_mode=3,                            # Assuming '1' is for netted positions, which is typical for individual traders
    price_tick_step=1,                          # Minimum price movement for Bitcoin, appropriate given its value
)
exchange_settings

ExchangeSettings(asset_tick_step=3, leverage_mode=1, leverage_tick_step=2, limit_fee_pct=0.0003, market_fee_pct=0.0006, max_asset_size=100.0, max_leverage=150.0, min_asset_size=0.001, min_leverage=1.0, mmr_pct=0.004, position_mode=3, price_tick_step=1)

In [172]:
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,    # Increase position size based on risk percentage of account size
    leverage_strategy_type=LeverageStrategyType.Dynamic,                    # Leverage is dynamically adjusted based on account size
    pg_min_max_sl_bcb="min",                                                # Stop loss is based on the minimum of the last two candle bodies
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,              # Stop loss is based on the candle body
    sl_to_be_bool=False,                                                    # Stop loss is not based on the candle body
    starting_bar=50,                                                        # Start the strategy after 50 bars
    starting_equity=1000.0,                                                 # Start with $1000
    static_leverage=None,                                                   # Leverage is dynamically adjusted based on account size
    tp_fee_type="limit",                                                    # Take profit is based on the limit fee
    tp_strategy_type=TakeProfitStrategyType.RiskReward,                     # Take profit is based on the risk-reward ratio
    trail_sl_bool=False,                                                    # Trailing stop loss is not used
    z_or_e_type=None,                                                       # Z-score or entropy is not used
)
static_os

StaticOrderSettings(increase_position_type=3, leverage_strategy_type=0, pg_min_max_sl_bcb='min', sl_strategy_type=1, sl_to_be_bool=False, starting_bar=50, starting_equity=1000.0, static_leverage=None, tp_fee_type='limit', tp_strategy_type=0, trail_sl_bool=False, z_or_e_type=None)

In [177]:
dos_arrays = DynamicOrderSettingsArrays(
    account_pct_risk_per_trade=np.array([12]),
    max_trades=np.array([0]),
    risk_reward=np.array([2, 3, 4, 5, 6, 7, 8, 9, 10]),
    sl_based_on_add_pct=np.array([0.1, 0.25, 0.5]),
    sl_based_on_lookback=np.array([10]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)
dos_arrays

DynamicOrderSettingsArrays(max_trades=array([0]), account_pct_risk_per_trade=array([12]), risk_reward=array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]), sl_based_on_add_pct=array([0.10, 0.25, 0.50]), sl_based_on_lookback=array([10]), sl_bcb_type=array([3]), sl_to_be_cb_type=array([6]), sl_to_be_when_pct=array([0]), trail_sl_bcb_type=array([3]), trail_sl_by_pct=array([0.50, 1.00]), trail_sl_when_pct=array([1, 2]))

In [178]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=macd_strat,
)

TypeError: 'NoneType' object is not subscriptable

In [180]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

NameError: name 'backtest_results' is not defined

In [179]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=macd_strat,
    dos_index=12,
    ind_set_index=2,
    plot_results=True,
    logger_bool=True,
)

DynamicOrderSettings(
    max_trades = 0,
    account_pct_risk_per_trade = 0.12,
    risk_reward = 3.0,
    sl_based_on_add_pct = 0.001,
    sl_based_on_lookback = 10,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.005,
    trail_sl_when_pct = 0.01,
)


c:\Coding\qfstrat\qfstrat\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [181]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
